In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [3]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [4]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [5]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [6]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [7]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [8]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [9]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat artists with other variables
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [23]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [24]:
mlp.score(X, Y)

0.69512218774243595

In [25]:
Y.value_counts()/len(Y)

Prints & Illustrated Books    0.523681
Photography                   0.225233
Architecture & Design         0.112316
Drawings                      0.104073
Painting & Sculpture          0.034697
Name: Department, dtype: float64

In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

array([ 0.59165172,  0.68277902,  0.47539155,  0.39874891,  0.44796819])

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization paramater that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a binary function ('relu', or 'rectified linear unit function'). In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentaiton](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [29]:
# Your code here. Experiment with hidden layers to build your own model.

# Split the data into four equally-sized samples. First we break it in half:
from sklearn.model_selection import train_test_split
X_half1, X_half2, Y_half1, Y_half2 = train_test_split(
    X,
    Y,
    test_size=0.5,
    random_state=None)
X1, X2, Y1, Y2 = train_test_split(
    X_half1, Y_half1,
    test_size=0.5,
    random_state=None)
X3, X4 , Y3, Y4 = train_test_split(
    X_half2, Y_half2,
    test_size=0.5,
    random_state=None)

In [31]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.588712179984
[ 0.60604886  0.54236959  0.41272304  0.47371484  0.53725262]


In [32]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(100,4,5))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.523351435221
[ 0.52326483  0.5233663   0.54499612  0.54684772  0.52347691]


In [33]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.629868114818
[ 0.54575417  0.54702346  0.59678045  0.53055286  0.60787738]


In [34]:
# solver = 'lbfgs'
# Note: The default solver ‘adam’ works pretty well on relatively large datasets (with thousands of training samples or more) 
# in terms of both training time and validation score. For small datasets, however, ‘lbfgs’ can converge faster and 
# perform better.
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10),solver = 'lbfgs' )
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.527734678045
[ 0.53198914  0.5233663   0.52327386  0.52531523  0.52347691]


In [35]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.640147401086
[ 0.62718108  0.62323056  0.60744763  0.60795344  0.58129608]


In [36]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10,10))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.634600465477
[ 0.61942613  0.60034904  0.55042669  0.55053346  0.55490881]


In [37]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.640535298681
[ 0.61632416  0.61993407  0.63906129  0.58525703  0.41598758]


In [38]:
# Establish and fit the model
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100))
mlp.fit(X1, Y1)
print(mlp.score(X1, Y1))
from sklearn.model_selection import cross_val_score
print(cross_val_score(mlp, X1, Y1, cv=5))

0.606904577192
[ 0.62718108  0.57029281  0.59193173  0.60155189  0.32460225]


In [39]:
# so far the 20,20, 20 looks the best - 0.64

In [43]:
# do for the whole dataset
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))
mlp.fit(X, Y)
print(mlp.score(X, Y))
y_pred = mlp.predict(X)

0.694133048875


In [44]:
X['dept_actual'] = Y

In [45]:
X['dept_predict'] = y_pred

In [47]:
pd.crosstab(Y, y_pred)

col_0,Architecture & Design,Drawings,Painting & Sculpture,Photography,Prints & Illustrated Books
Department,,,,,
Architecture & Design,6012,68,308,1671,3523
Drawings,1026,1400,389,1808,6109
Painting & Sculpture,986,76,1445,196,875
Photography,423,73,140,16115,6475
Prints & Illustrated Books,1179,293,185,5738,46607


In [48]:
# Display our results.
print("Train:Number of mislabeled points out of a total {} points : {}".format(
    X.shape[0],
    (Y != y_pred).sum()
))

Train:Number of mislabeled points out of a total 103120 points : 31541


In [49]:
#About 70% correct
# biggest error was drawings vs. prints & illustrated books...
# this dataset is so boring